In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.request
import cv2
import requests
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

import os
# import re

In [2]:
def save_photos(df): #Download images
    for ind in df.index:
        image_url_list = df['Image_url'][ind]
        stone_name = df['Name'][ind]
        if(stone_name.find(' Opal') != -1 or stone_name.find(' Diamond') != -1 or stone_name.find(" Bone") != -1):continue
        FilePath = 'Test_for_Hezi/'
        for i,image_url in enumerate(image_url_list):
            newpath = FilePath + stone_name +'/'
            if not os.path.exists(newpath): os.makedirs(newpath)
            image_url = 'https://www.mindat.org/'+image_url
            response = requests.get(image_url)
            image_name = stone_name +'_num_'+str(i) + '.png' 
            with open(newpath +image_name, 'wb') as file:
                file.write(response.content)
            img = cv2.imread(image_name)


def get_photos(driverf,stone_name): # Get photo's URLs
    time.sleep(1)   
    img_list = []
    try:
        driverf.find_element_by_css_selector("input[id='searchbox']").send_keys(Keys.CONTROL + 'a', Keys.BACKSPACE)
        driverf.find_element_by_css_selector("input[id='searchbox']").send_keys(stone_name)
        driverf.find_element_by_css_selector("input[id='searchbox']").send_keys(Keys.ENTER)
        
        SCROLL_PAUSE_TIME = 0.5
        last_height = driverf.execute_script("return document.body.scrollHeight")
        while True:
            driverf.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height =  driverf.execute_script("return document.body.scrollHeight")
            if(new_height==last_height): break
            last_height = new_height
    except ElementClickInterceptedException:
        return img_list
    except NoSuchElementException:
        pass
    except TypeError:
        pass
    soup = BeautifulSoup(driverf.page_source, 'html.parser')
    try:
        table = soup.find("div",{"id":"mainwrap"})
        images_table = table.find("div",{"id":"photoscroll"})
        images = images_table.find_all("a")

        for img in images:
            img_list.append(img.find('img')['src'])
    except NoSuchElementException: pass
    except AttributeError: pass
    except TypeError: pass
    return img_list

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driverf = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://geology.com/gemstones/') #Gets the most popular gemstone's names
driverf.get('https://www.mindat.org/photoscroll.php') # Gets the gemstone's images

soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup.find("div",{"class":"right"})
Stones_list = []

href_list = table.find_all('a')
driver.close()
i=0
for href in href_list:
    time.sleep(1)
    data_dict = {}
    Name = href.text
    if(Name.find("K2 Granite")!=-1): Name = "Granite"
    if(Name.find(' Opal') != -1 or Name.find(' Diamond') != -1 or Name.find("Bone") != -1):continue
    data_dict['Name'] = Name
    image_list = get_photos(driverf, Name)
    image_list = list(dict.fromkeys(image_list))
    data_dict['Image_url'] = image_list
    Stones_list.append(data_dict)
    if(i>1): break
    i+=1


driverf.close()    
df = pd.DataFrame(Stones_list)
save_photos(df)



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\noamb\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache
C:\Users\noamb\AppData\Local\Temp/ipykernel_10980/1629477268.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\noamb\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache
C:\Users\noamb\AppData\Local\Temp/ipykernel_10980/1629477268.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driverf = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\noamb\AppData\Local\Temp/ipykernel_10980/1559804279.py:22: DeprecationWarning: f

TypeError: 'NoneType' object is not subscriptable